In [204]:
def areaComp(urlzooplaWeb):
    pattern_attribute = re.compile(r'{"__typename":"Attributes","bathrooms":(.*?),"bedrooms":(.*?),"floorAreaSqM":(.*?),"livingRooms":(.*?),"tenure":(.*?),"propertyType":"(.*?)"}')

    # urlzooplaWeb = 'https://www.zoopla.co.uk/property/252-holly-lane/erdington/birmingham/b24-9lh/517035'
    headZooplaWeb = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    response_1st = requests.get(urlzooplaWeb, headers=headZooplaWeb)

    request = urllib.request.Request(response_1st.url, headers=headZooplaWeb)
    response_2rd = urllib.request.urlopen(request)
    html = response_2rd.read().decode('utf-8')

    soup_str = str(BeautifulSoup(html, 'html.parser'));
    attribute_str = pattern_attribute.findall(soup_str)

    floorArea = float("nan")
    if np.size(attribute_str) != 0 and attribute_str[0][2] != 'null':
        floorArea = float(attribute_str[0][2])
    return floorArea

In [202]:
import urllib
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import requests

url = "https://zoopla.p.rapidapi.com/house-prices/estimate"
houseNumberPattern = re.compile(r'\b[0-9]{1,6}\b')
headers = {
	"X-RapidAPI-Key": "76325c0abamshe24a0bd685d4898p1ed9b3jsn63784230fa57",
	"X-RapidAPI-Host": "zoopla.p.rapidapi.com"
}

price_lower = []
price_middle = []
price_higher = []
address = []
floorArea = []
type = []
bathroom_num = []
bedroom_num = []
livingroom_num = []
lastSoldYear = []
longitude = []
latitude = []
house_number = []

beginIdx = 276
endIdx = 320
indexDif = 2
house_number_ref = np.arange(beginIdx, endIdx+1, indexDif)

for page_number in np.arange(1,2):
	querystring = {"area":"Holly Lane, Erdington, Birmingham, B24 9LH", "page_number":str(page_number),"page_size":"100"}
	responseZooplaApi = requests.request("GET", url, headers=headers, params=querystring)

	properties = responseZooplaApi.json()['property']
	numOfHouses = np.size(properties)
	print(numOfHouses)

	if numOfHouses == 0:
		break
	
	for i in np.arange(0, numOfHouses):
		address_str = properties[i]['address']
		houseNumber = int(houseNumberPattern.findall(address_str)[0])

		if houseNumber in house_number_ref:
			# print("House Idx:", houseNumber)

			price_middle.append(properties[i]['estimate_value'])
			price_lower.append(properties[i]['estimate_value_lower'])
			price_higher.append(properties[i]['estimate_value_upper'])

			type.append(properties[i]['property_type'])
			address.append(properties[i]['address'])

			bathroom_num.append(properties[i]['num_baths'])
			bedroom_num.append(properties[i]['num_beds'])
			livingroom_num.append(properties[i]['num_recepts'])
			if properties[i]['last_sale_date'] == None:
				lastSoldYear.append(float('nan'))
			else:
				lastSoldYear.append(properties[i]['last_sale_date'][0:4])

			latitude.append(float(properties[i]['latitude']))
			longitude.append(float(properties[i]['longitude']))

			house_number.append(houseNumber)

			urlzooplaWeb = properties[i]['refine_estimate_url']
			if urlzooplaWeb == None:
				floorArea.append(float('nan'))
			else:
				floorArea.append(areaComp(urlzooplaWeb))


postCodePattern = re.compile(
    r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b')
postCode = postCodePattern.findall(address[0])
print(postCode[0])
print(price_lower[0])

roadNamePattern = re.compile(r'\d{1,3} (.*?),')
roadName = roadNamePattern.findall(address[0])
roadName[0]
print(roadName[0])
address

20
B24 9LH
233000
Holly Lane


['276 Holly Lane, Erdington, Birmingham B24 9LH',
 '278 Holly Lane, Erdington, Birmingham B24 9LH',
 '280 Holly Lane, Erdington, Birmingham B24 9LH',
 '282 Holly Lane, Erdington, Birmingham B24 9LH',
 '284 Holly Lane, Erdington, Birmingham B24 9LH',
 '286 Holly Lane, Erdington, Birmingham B24 9LH',
 '288 Holly Lane, Erdington, Birmingham B24 9LH',
 '290 Holly Lane, Erdington, Birmingham B24 9LH',
 '292 Holly Lane, Erdington, Birmingham B24 9LH',
 '294 Holly Lane, Erdington, Birmingham B24 9LH',
 '296 Holly Lane, Erdington, Birmingham B24 9LH']

In [203]:
raw_data = pd.DataFrame({"Low": price_lower, "Estimated price": price_middle, "High": price_higher,
                        "Address": address, 'House number': house_number, 
                        "Longitude": longitude, "Latitude": latitude, "Last Sold": lastSoldYear,
                        "Floor Area": floorArea, "Property Type": type,
                        "Bedrooms": bedroom_num, "Bathrooms": bathroom_num, "Livingrooms": livingroom_num})
# raw_data.to_csv("zoopla Geo data/"+postCode[0]+" price data Geo.csv", index=True, header=True)
raw_data

,Low,Estimated price,High,Address,House number,Longitude,Latitude,Last Sold,Floor Area,Property Type,Bedrooms,Bathrooms,Livingrooms
0,233000,245000,257000,"276 Holly Lane, Erdington, Birmingham B24 9LH",276,-1.823330,52.522424,2018,103.0,Semi-Detached House,3.0,1.0,2.0
1,222000,247000,272000,"278 Holly Lane, Erdington, Birmingham B24 9LH",278,-1.822680,52.522230,NaN,NaN,None,NaN,NaN,NaN
2,236000,248000,261000,"280 Holly Lane, Erdington, Birmingham B24 9LH",280,-1.822680,52.522230,2017,90.0,Semi-Detached House,3.0,1.0,1.0
3,242000,255000,268000,"282 Holly Lane, Erdington, Birmingham B24 9LH",282,-1.822664,52.522277,2022,117.0,Semi-Detached House,3.0,1.0,2.0
4,234000,246000,258000,"284 Holly Lane, Erdington, Birmingham B24 9LH",284,-1.822680,52.522230,2020,98.0,Semi-Detached House,3.0,1.0,2.0
5,238000,251000,263000,"286 Holly Lane, Erdington, Birmingham B24 9LH",286,-1.822664,52.522277,2017,110.0,Semi-Detached House,3.0,1.0,2.0
6,223000,248000,273000,"288 Holly Lane, Erdington, Birmingham B24 9LH",288,-1.822664,52.522277,NaN,NaN,None,NaN,NaN,NaN
7,238000,251000,264000,"290 Holly Lane, Erdington, Birmingham B24 9LH",290,-1.822664,52.522277,NaN,111.0,Semi-Detached House,3.0,NaN,2.0
8,234000,246000,259000,"292 Holly Lane, Erdington, Birmingham B24 9LH",292,-1.822664,52.522277,2015,98.0,Semi-Detached House,3.0,1.0,2.0
9,245000,258000,271000,"294 Holly Lane, Erdington, Birmingham B24 9LH",294,-1.822700,52.522010,2012,100.0,Semi-Detached House,4.0,2.0,2.0
